### Imports

In [ ]:
import os
import time
import random
import glob
import pandas as pd
import geopandas as gpd
import osmnx as ox
from pathlib import Path

### Projektordner

In [5]:
PROJECT_DIR = "scs_osm_project"   # hier kommen die gpkg-Ordner hin
CSV_PATH = "islands_master.csv"          # CSV-Masterdatei

# Ordnernamen für die gpkg-Dateien
CACHE_DIR  = os.path.join(PROJECT_DIR, "cache_gpkg_new")
MASTER_DIR = os.path.join(PROJECT_DIR, "master_gpkg_new")

# Erstelle die beiden Projektordner , falls sie noch nicht existieren.
# exist_ok=True verhindert einen Fehler, wenn die Ordner bereits vorhanden sind.
for folder in [CACHE_DIR, MASTER_DIR]:
    os.makedirs(folder, exist_ok=True)

print("Folders:")
print("CACHE :", CACHE_DIR)
print("MASTER:", MASTER_DIR)

Folders:
CACHE : scs_osm_project\cache_gpkg_new
MASTER: scs_osm_project\master_gpkg_new


### CSV laden + korrekte bbox erzeugen

In [ ]:
# CSV-Datei laden
df = pd.read_csv(CSV_PATH, sep=";")

# Liste aller Spalten, die wir aus der CSV brauchen
needed_colums = ["id","island","island_group","country","north","south","west","east"]

# Hier wird geprüft, ob alle benötigten Spalten in der CSV-Datei vorhanden sind, es wird True oder False geprintet
if set(needed_colums).issubset(df.columns):
    print("Alle benötigten Spalten sind vorhanden.")
else:
    print("Mindestens eine benötigte Spalte fehlt.")


# Kopie erstellen nur mit den benötigten Spalten, gegeben durch die Liste "needed"
df = df[needed_colums].copy()

# Koordinaten-Spalten in numerische Werte umwandeln, damit die Werte nicht als String interpretiert werden
# wir haben "errors='coerce'" gesetzt, damit ungültige Werte zu NaN werden und kein Fehler ausgelöst wird
for c in ["north", "south", "west", "east"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")


# Printen, ob alle Bounding Boxes vollständig sind
if df[["north","south","west","east"]].notna().all(axis=1).all():
    print("Alle Bounding Boxes sind vollständig.")
else:
    print("Mindestens eine Bounding Box ist unvollständig.")


# OSMnx erwartet die Bounding Boxen als Tupel (west, south, east, north), daher erstellen wir eine neue Spalte "bbox" mit diesen Tupeln in der richtigen Reihenfolge
df["bbox"] = list(zip(df["west"], df["south"], df["east"], df["north"]))


df.head()


Alle benötigten Spalten sind vorhanden.
Alle Bounding Boxes sind vollständig.


,id,island,island_group,country,north,south,west,east,bbox
0,1,Cuarteron Reef,Spratly,China,8.890,8.84,112.80,112.87,"(112.8, 8.84, 112.87, 8.89)"
1,2,Fiery Cross Reef,Spratly,China,9.580,9.53,112.86,112.92,"(112.86, 9.53, 112.92, 9.58)"
2,3,Gaven Reefs,Spratly,China,10.220,10.19,114.21,114.24,"(114.21, 10.19, 114.24, 10.22)"
3,4,Hughes Reef,Spratly,China,9.927,9.90,114.48,114.51,"(114.48, 9.9, 114.51, 9.927)"
4,5,Johnson Reef,Spratly,China,9.750,9.69,114.26,114.30,"(114.26, 9.69, 114.3, 9.75)"


### Overpass-Einstellungen

In [ ]:
# Dictionary mit OSM-Tag-Abfragen (Keys = "Feature-Typ", Werte = OSM-Tags)
QUERIES = {
    "runways": {"aeroway": "runway"},
    "helipads": {"aeroway": ["helipad", "heliport"]},
    "towers": {"man_made": "tower"},
    "marinas": {"leisure": "marina"},
    "ferry_terminals": {"amenity": "ferry_terminal"},
}


# Wir haben OSMMnx und nicht ohsome benutzt, da wir nur die aktuellen OSM-Daten extrahieren möchten

# ----------------------
# OSMxn (Overpass) Einstellungen [ChatGPT 1]
# ----------------------


# use_cache=True: aktiviert das Caching von Abfragen, um wiederholte Anfragen zu beschleunigen
ox.settings.use_cache = True

# log_console=True: aktiviert die Protokollierung von Aktivitäten in der Konsole, hilfreich zum Debuggen
ox.settings.log_console = True

# overpass_settings: bestimmt das Ausgabeformat (in unserem Fall json) und den Timeout für Overpass-Abfragen (180 Sekunden darf der Server brauchen)
ox.settings.overpass_settings = "[out:json][timeout:180]"

# URL zum Overpass-Server, der die OSM-Abfragen beantwortet
ox.settings.overpass_url = "https://overpass.kumi.systems/api/interpreter"



# Rate-Limit/Pausen zwischen Abfragen, viele Abfragen sollen wohl vom Overpass-Server gedrosselt werden
BASE_SLEEP = 6 # Grund-Wartezeit in Sekunden zwischen zwei Overpass-Abfragen
JITTER = 2 # Jitter ist eine zusätzliche Wartezeit, die zufällig zwischen 0 und JITTER Sekunden variiert, um die Abfragen unregelmäßiger zu machen (regelmäßige Abfragen mag der Server nicht)


# Funktion, welche eine zufällige Zahl zwischen 0 und JITTER generiert und diese zur BASE_SLEEP addiert, um die Wartezeit ("sleep", aus dem Modul "time") zu bestimmen
def safe_sleep():
    time.sleep(BASE_SLEEP + random.uniform(0, JITTER))


# Checken, wie viele Inseln abgefragt werden
print("Islands to process:", len(df))


Islands to process: 70


### Hilfsfunktionen

In [8]:
# Hilfsfunktion, welche Strings (z.B. Inselnamen) in "saubere" Strings umwandelt, indem alle nicht-alphanumerischen Zeichen durch Unterstriche ersetzt werden
def slug(s: str) -> str:
    return "".join(ch if ch.isalnum() else "_" for ch in str(s)).strip("_")


# Erstellt den Pfad zur Cache-Datei (Geopackage) für eine bestimmte Insel und einen bestimmten Feature-Typ
# Der Dateiname besteht aus dem f-string: "Insel-ID (3-stellig, mit führenden Nullen)__Inselname (sauber)__Feature-Typ.gpkg"
# Wir benutzen doppelte Unterstriche __ als Trenner, damit Inselnamen mit einfachen Unterstrichen _ (Ersatz für Leerzeichen) keine Probleme machen
# Layer-Key ist der Feature-Typ (z.B. "runways", "towers", etc.)
def cache_path(island_id, island_name, layer_key):
    return os.path.join(CACHE_DIR, f"{int(island_id):03d}__{slug(island_name)}__{layer_key}.gpkg")


# Lädt die OSM-Features über ox.features_from_bbox innerhalb einer Bounding Box mit den angegebenen Tags und gibt ein GeoDataFrame zurück
# OSMnx speichert die OSM-ID im Index, reset_index() wandelt den Index in normale Spalten um,
# damit alle Informationen sauber im GeoPackage gespeichert und später einfacher weiterverarbeitet werden können
def fetch_features_bbox(bbox, tags):
    return ox.features_from_bbox(bbox=bbox, tags=tags).reset_index()


### Inseln downloaden

In [9]:
# Funktion lädt die passenden OSM-Daten für jeden Feature-Typ in QUERIES und speichert sie als GeoPackage im Cache-Ordner
def download_island_to_cache(island_id, island_name, country, island_group, bbox, overwrite=False):
    
    # Printet Infos zur momentan abgefragten Insel
    print(f"\n{int(island_id):03d} | {island_name} ({country}) bbox={bbox} ")

    # Schleife geht alle Eintrage in QUERIES durch für jede Insel
    for layer_key, tags in QUERIES.items():
        
        # Ausgabe-Pfad der Cache-Datei für die aktuelle Insel und den aktuellen Feature-Typ mithilfe der Hilfsfunktion cache_path
        out_path = cache_path(island_id, island_name, layer_key)

        # Wenn die Overwrite=False und die Cache-Datei bereits existiert, wird sie geladen und der Download übersprungen
        if (not overwrite) and os.path.exists(out_path):
            try:
                g_cached = gpd.read_file(out_path)
                print(f"  {layer_key}: cached ({len(g_cached)})")
                safe_sleep()
                continue
            except Exception:
                # falls Cache-Datei kaputt ist, wird sie neu erzeugt
                pass

        # Versuche, die OSM-Features für die aktuelle bounding box und die angegebenen Tags herunterzuladen
        try:
            # Startzeit merken, um die Dauer des Downloads zu messen
            t0 = time.time()

            # OSM-Features innerhalb der Bounding Box abrufen (z. B. runways, helipads, etc.)
            gdf = fetch_features_bbox(bbox, tags)

            # Vergangene Zeit seit dem Start berechnen
            dt = time.time() - t0

            # Falls keine Objekte gefunden wurden, wird nichts gespeichert und direkt mit dem nächsten Feature-Typ weitergemacht
            if gdf is None or len(gdf) == 0:
                print(f"  {layer_key}: no features found (0) in {dt:.1f}s")
                safe_sleep()
                continue

            print(f"  {layer_key}: downloaded {len(gdf)} in {dt:.1f}s")

            # Metadaten aus OSM für jedes Objekt hinzufügen
            gdf["src_island_id"] = int(island_id)
            gdf["src_island"] = island_name
            gdf["src_country"] = country
            gdf["src_group"] = island_group
            gdf["src_layer"] = layer_key
            # Geodataframe als GeoPackage speichern
            gdf.to_file(out_path, driver="GPKG")
            print(f"  {layer_key}: saved -> {os.path.basename(out_path)}")


        # Wenn Fehler auftreten werden diese geprintet, aber das Script läuft weiter
        except Exception as e:
            print(f"  {layer_key}: download error (skipped) -> {repr(e)}")

        # Hilfsfunktion aufrufen, um den Overpass-Server nicht zu überlasten
        safe_sleep()


### RUN: alle Inseln downloaden (Cache füllen)

In [ ]:
# overwrite=False -> nutzt Cache, lädt nicht neu
# overwrite=True  -> lädt alles neu
overwrite = True


# ---------
# Da der Download nicht für alle Inseln erfolgreich war, mussten wir nachträglich eine Funktion integrieren, um die fehlenden Inseln erneut herunterzuladen
# Dies geschah mit Hilfe von KI-Unterstützung (ChatGPT 3)
# ---------
# Downloadbereich einstellen (welche Inseln herunterladen?), Werte inklusive (ChatGPT 3)
# END_ID = None -> bis zum Ende laufen lassen
# Notiz an Andy: Wenn du alle Inseln laden möchtest, lade nur ein paar Inseln herunter, denn der Download aller Inseln hat mehrere Stunden gedauert
START_ID = 1
END_ID   = None

# Sortiert die CSV-Tabell nach id, damit die Inseln in sauberer Reihenfolge abgearbeitet werden
df_run = df.sort_values("id").copy()

# ab START_ID laufen lassen
df_run = df_run[df_run["id"] >= START_ID].copy()

# bis END_ID laufen lassen, falls gesetzt
if END_ID is not None:
    df_run = df_run[df_run["id"] <= END_ID].copy()

# total speichert die Anzahl der Inseln, die heruntergeladen werden sollen
total = len(df_run)

# Schleife, welche Zeile Zeile durch df_run iteriert und die Inseln herunterlädt
# enumrate wird genutzt fürs Fortschritts-Printing
for i, row in enumerate(df_run.itertuples(index=False), start=1):

    # Printet den Fortschritt
    print(f"\n[{i}/{total}] Processing: id={row.id} | {row.island}")

    # Ruft die Funktion auf, um die Insel herunterzuladen
    # Wichtig: Das Processen dauert sehr lange, wenn keine Features gefunden werden (download error)
    # Wir hatten Download-Zeiten von ca. 2-3 Minuten pro Insel
    download_island_to_cache(
        island_id=row.id,
        island_name=row.island,
        country=row.country,
        island_group=row.island_group,
        bbox=row.bbox,
        overwrite=overwrite
    )

print("\nDone downloading selected islands.")


[1/1] Processing: id=1 | Cuarteron Reef

001 | Cuarteron Reef (China) bbox=(112.8, 8.84, 112.87, 8.89) 
  runways: download error (skipped) -> InsufficientResponseError('No matching features. Check query location, tags, and log.')
  helipads: downloaded 3 in 0.1s
  helipads: saved -> 001__Cuarteron_Reef__helipads.gpkg
  towers: downloaded 5 in 0.1s
  towers: saved -> 001__Cuarteron_Reef__towers.gpkg
  marinas: downloaded 1 in 0.1s
  marinas: saved -> 001__Cuarteron_Reef__marinas.gpkg
  ferry_terminals: download error (skipped) -> InsufficientResponseError('No matching features. Check query location, tags, and log.')

Done downloading selected islands.


### Pro Feature-Typ eine Masterdatei

In [ ]:
# in Cache_gpkg werden die OSM-Features pro Insel und Feature-Typ gespeichert
# wir möchten aber pro Feature-Typ eine Master-Datei mit allen Inseln zusammenführen
# diese Funktion liest alle Cache-Dateien ein, fasst sie pro Feature-Typ zusammen und speichert sie im Master-Ordner
def build_master_global():
    
    # os.path.join(CACHE_DIR, "*.gpkg") baut einen Pfad, der alle GeoPackage-Dateien im Cache-Ordner umfasst
    # glob.glob () sucht nach allen GeoPackage-Dateien im Cache-Ordner
    # Wir speichern in der Variable files alle .gpkgnach allen GeoPackage-Dateien (*.gpkg) gesucht
    # sorted() sortiert die Liste der Dateien alphabetisch
    # Ergebnis: files ist eine Liste von Dateipfaden zu den Cache-Dateien (GeoPackages)
    files = sorted(glob.glob(os.path.join(CACHE_DIR, "*.gpkg")))
    
    #Gibt aus, wie viele Cache-Dateien gefunden wurden
    print("Total cache files:", len(files))
    if len(files) == 0:
        print("No cache files found.")
        return

    # Wir erstellen ein leeres Dictionary, in dem wir später die Daten nach Feature-Typen (Layer-Keys) gruppieren
    buckets = {}

    # Schleife über alle Keys in QUERIES
    for layer_key in QUERIES.keys():
        
        # Für jeden Layer-Key wird eine leere Liste angelegt
        # Hier werden später die GeoDataFrames gesammelt, die zu diesem Typ gehören
        buckets[layer_key] = []

    # Schleife über alle gefundenen Cache-Dateien
    # fp ist jeweils der filepath zu einer einer Cache-GPKG
    for fp in files:

        # os.path.basename(fp) nimmt nur den Dateinamen (ohne Ordner) -> aus .../026__X__helipads.gpkg wird 026__X__helipads.gpkg
        # os.path.splitext(...)[0] entfernt die Endung .gpkg -> aus 026__X__helipads.gpkg wird 026__X__helipads
        # name ist also der Basisname der Datei
        name = os.path.splitext(os.path.basename(fp))[0]

        # Teilt den Dateinamen an __ auf -> 026__Fiery_Cross_Reef__helipads wird zu: ["026", "Fiery_Cross_Reef", "helipads"]
        parts = name.split("__")

        # Basic Sicherheitstest, um zu prüfen, ob der Dateiname das erwartete Format hat (3 Teile)
        # Falls nicht, wird die Datei übersprungen
        if len(parts) != 3:
            continue

        # Der Layer-Key ist der letzte Teil des Dateinamens (z.B. "helipads", "runways", etc.)
        layer_key = parts[-1]

        # GeoPackage mit GeoPandas einlesen und als GeoDataFrame speichern
        g = gpd.read_file(fp)

        # Wenn die Datei leer ist, wird sie übersprungen
        if len(g) == 0:
            continue
        

        # Das gelesene GeoDataFrame g wird der passenden Liste im buckets-Dictionary hinzugefügt
        buckets[layer_key].append(g)



    # Für jeden Feature-Typ (Layer-key) alle Daten zusammenführen und in einer Msater-Datei speichern
    for layer_key in buckets.keys():

        # gdfs ist die Liste aller GeoDataFrames, die zu diesem Typ gehören
        gdfs = buckets[layer_key]

        # Hier bauen wir den Pfad zur Master-Datei für den aktuellen Layer-Key
        out_master = os.path.join(MASTER_DIR, f"master__{layer_key}.gpkg")

        # Wenn keine Daten für diesen Layer-Key gefunden wurden, wird eine Meldung ausgegeben und weiter zum nächsten Layer-Key gegangen
        if len(gdfs) == 0:
            print("Master", layer_key, ": no data")
            continue

        # pd.concat() hängt alle GeoDataFrames in gdfs untereinander zu einem großen zusammen
        # ignore_index=True macht einen neuen Index 0,1,2,… (statt alte Indizes zu behalten)
        # gpd.GeoDataFrame() sorgt dafür, dass das Ergebnis wieder ein GeoDataFrame ist
        # geometry="geometry" sagt, dass die Spalte geometry Geometrien enthält
        # crs=gdfs[0].crs übernimmt das Koordinatensystem vom ersten GeoDataFrame
        g_all = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), geometry="geometry", crs=gdfs[0].crs)

        # Weiterer Sicherheitsschritt: 
        # Falls sich irgendwo Duplikate (wenn ein OSM-Objekt (osmid) im gleichen Layer (src_layer mehrfach vorkommt)) eingeschlichen haben, werden diese entfernt 
        # mit der Funktion drop_duplicates()
        # Nur wenn osmid existiert, können wir Duplikate identifizieren
        if "osmid" in g_all.columns:
            g_all = g_all.drop_duplicates(subset=["osmid", "src_layer"])
            print("Duplicates removed")

        else:
            print("No duplicates removed")

        # Speichert das zusammengeführte GeoDataFrame (g_all) als Geopackage im vorher definierten Master-Ordner (out_master)
        # driver="GPKG" sagt: Dateiformat = GeoPackage
        g_all.to_file(out_master, driver="GPKG")

        # Printet kurze Meldung, welcher Master-Layer gespeichert wurde, wie die Datei heißt und wie viele Objekte darin sind
        print("Master", layer_key, ": saved", os.path.basename(out_master), "(", len(g_all), ")")

build_master_global()


Total cache files: 3
Master runways : no data
No duplicates removed
Master helipads : saved master__helipads.gpkg ( 3 )
No duplicates removed
Master towers : saved master__towers.gpkg ( 5 )
No duplicates removed
Master marinas : saved master__marinas.gpkg ( 1 )
Master ferry_terminals : no data


### Alle Geometrien in Punkte umwandeln für Heatmaps

In [12]:
#----------------------
# Da wir in ArcGIS-Online Heatmaps aus Punktdaten erstellen wollen, müssen alle Geometrien in Punkte umgewandelt werden
#----------------------


# MASTER_DIR ist in unserem Script bisher ein normaler String-Pfad (kommt von os.path.join(...)),
# z.B. "scs_osm_project/master_gpkg_test".
# Für Funktionen wie .glob(...) und den "/"-Operator zum Pfad-Zusammenbauen brauchen wir aber ein Path-Objekt.
# Path(MASTER_DIR) wandelt den String in ein Path-Objekt um.
MASTER_DIR_PATH = Path(MASTER_DIR)

# .glob("master__*.gpkg") sucht im Ordner MASTER_DIR_PATH alle Dateien,
# deren Name mit "master__" beginnt und auf ".gpkg" endet (also alle Master-GeoPackages).
# sorted(...) sortiert die gefundenen Dateien, damit die Reihenfolge immer gleich ist.
# Ergebnis: master_files ist eine Liste von Path-Objekten zu den gefundenen Dateien
master_files = sorted(MASTER_DIR_PATH.glob("master__*.gpkg"))


# Neue Liste, mit den Dateien, die wir umwandeln wollen 
clean_files = []

# Wir gehen jede Datei in master_files durch
for p in master_files:

    # Dateiname der aktuellen Datei holen
    name = p.name

    # Wir filtern die Dateien heraus, die bereits als _POINTS_FOR_AGOL oder _PUBLISH_READY gespeichert wurden
    if "_POINTS_FOR_AGOL" in name:
        continue
    if "_PUBLISH_READY" in name:
        continue

    # Wenn die Datei nicht übersprungen wurde, wird sie der Liste clean_files hinzugefügt
    clean_files.append(p)



# Listet auf, wieviele und welche Dateien umgewandelt werden
print("Umzuwandelnde Dateien:", len(clean_files))
for p in clean_files:
    print(" -", p.name)



# Hilfsfunktion, die eine Geometrie in einen Punkt umwandelt
def geom_to_point(geom):
    
    # Wenn keine Geometrie vorhanden ist, nichts zurückgeben
    if geom is None:
        return None

    # Wenn die Geometrie bereits ein Punkt ist, ändern wir nichts
    if geom.geom_type == "Point":
        return geom

    # alle anderen Geometrie-Typen in einen Punkt umwandeln
    # Man hätte auch centroid nehmen können, aber representative_point() liegt immer innerhalb des Objekts
    try:
        return geom.representative_point()
    except Exception:
        # Fallback: Schwerpunkt
        return geom.centroid


# Schleife über alle Dateien, die umgewandelt werden sollen
for input_gpkg in clean_files:

    # input_gpkg ist ein Dateipfad zur aktuellen Datei
    # .stem gibt den Dateinamen ohne Endung zurück (z.B. "master__towers")
    # replace("master__", "") entfernt den Prefix "master__", damit nur "towers" übrig bleibt
    layer_key = input_gpkg.stem.replace("master__", "")

    # Namen der Output-Datei bauen
    # "/" zum aneinanderhängen von Pfad-Teilen
    output_gpkg = MASTER_DIR_PATH / ("master__" + layer_key + "_POINTS_FOR_AGOL.gpkg")


    # Name des Layers im GeoPackage (Beispiel: "towers_points")
    layer_name = layer_key + "_points"


    # Zur Übersicht printen, welche Datei gerade verarbeitet wird
    print("\n---")
    print("INPUT :", input_gpkg.name)
    print("OUTPUT:", output_gpkg.name)

    # GeoPackage-Datei einlesen -> Ergebnis ist ein GeoDataFrame (Tabelle mit Geometrien)
    gdf = gpd.read_file(input_gpkg)

    # Zeilen entfernen, bei denen keine Geometrie vorhanden ist (NaN/None)
    gdf = gdf[gdf.geometry.notna()]
    gdf = gdf[~gdf.geometry.is_empty]


    # CRS auf WGS84 setzen
    if gdf.crs is None:
        # Falls kein CRS bekannt ist, setzen wir es
        gdf = gdf.set_crs(epsg=4326)
    else:
        # Wenn ein CRS vorhanden ist, rechnen wir die Geometrien in EPSG:4326 um
        gdf = gdf.to_crs(epsg=4326)



    # Neue Liste für die Punkt-Geometrien
    new_geoms = []

    # Geometrien in Punkte umwandeln mit for-schleife
    for geom in gdf.geometry:

        # Wir nutzen die gerade definierte Funktion geom_to_point, um die Geometrie in einen Punkt umzuwandeln und
        # sie dann new_geoms hinzuzufügen
        new_geoms.append(geom_to_point(geom))

    # Die Geometriespalte wird durch die neuen Punkt-Geometrien ersetzt
    gdf["geometry"] = new_geoms

    # Zur Sicherheit: Nach der Umwandlung nochmal prüfen und ungültige/ leere Geometrien entfernen
    gdf = gdf[gdf.geometry.notna()]
    gdf = gdf[~gdf.geometry.is_empty]

    # Anzahl der Features nach der Umwandlung printen
    print("Rows after conversion:", len(gdf))

    # GeoDataFrame als neues GeoPackage speichern
    gdf.to_file(output_gpkg, layer=layer_name, driver="GPKG")

    # Bestätigung ausgeben, dass die Datei gespeichert wurde
    print("Saved:", output_gpkg.name)

Umzuwandelnde Dateien: 3
 - master__helipads.gpkg
 - master__marinas.gpkg
 - master__towers.gpkg

---
INPUT : master__helipads.gpkg
OUTPUT: master__helipads_POINTS_FOR_AGOL.gpkg
Rows after conversion: 3
Saved: master__helipads_POINTS_FOR_AGOL.gpkg

---
INPUT : master__marinas.gpkg
OUTPUT: master__marinas_POINTS_FOR_AGOL.gpkg
Rows after conversion: 1
Saved: master__marinas_POINTS_FOR_AGOL.gpkg

---
INPUT : master__towers.gpkg
OUTPUT: master__towers_POINTS_FOR_AGOL.gpkg
Rows after conversion: 5
Saved: master__towers_POINTS_FOR_AGOL.gpkg


### Doppelpunkte in Unterstriche umwandeln in Feldnamen


In [ ]:
# ------------------------------------------------------------
# ArcGIS Online akzeptiert die im vorherigen Schritt erstellten GeoPackages nicht
# Es kommt die Fehlermeldung "failed" (ohne weitere Details) und die Punkte werden im Feature-Layer nicht angezeigt
# Die KI hat die folgenden mögliche Ursachen genannt: (ChatGPT 2)
# -> Ungültige Geometrien, Komplexe Spalten, ungültige Feldnamen, problematische Datentypen, falsches CRS
# Durch Trial and Error haben wir herausgefunden, dass ungültige Feldnamen (die Doppelpunkte in den Spaltennamen, die als OSM-Tags mitgeliefert werden, z.B. ref:ourairports) den Fehler verursachen
# Lösung: Wir ersetzen die Doppelpunkte in den Spaltennamen durch Unterstriche
# ------------------------------------------------------------


# Alle Dateien in MASTER_DIR_PATH finden, dem Format master__[beliebiger_Text]_POINTS_FOR_AGOL.gpkg folgen und alphabetisch sortieren
points_files = sorted(MASTER_DIR_PATH.glob("master__*_POINTS_FOR_AGOL.gpkg"))

# Wir printen, wie viele und welche Dateien gefunden wurden
print("Gefundene Dateien:", len(points_files))
for p in points_files:
    print(" -", p.name)

# mit einer for-schleife jede Datei durchgehen
for input_gpkg in points_files:

    # layer_key aus dem Dateinamen extrahieren
    layer_key = input_gpkg.stem.replace("master__", "").replace("_POINTS_FOR_AGOL", "")

    # Wir wollen eine neue Datei speichern, die auf _PUBLISH_READY.gpkg endet und nicht mehr auf _POINTS_FOR_AGOL.gpkg
    output_gpkg = input_gpkg.with_name(input_gpkg.name.replace("_POINTS_FOR_AGOL.gpkg", "_PUBLISH_READY.gpkg"))

    # Kurz printen, welche Datei gerade verarbeitet wird
    print("\nINPUT :", input_gpkg.name)
    print("OUTPUT:", output_gpkg.name)

    # Datei einlesen
    gdf = gpd.read_file(input_gpkg)

    # Hier ersetzen wir die Doppelpunkte in den Spaltennamen durch Unterstriche
    gdf = gdf.rename(columns=lambda c: c.replace(":", "_"))

    # Speichern (Layername = layer_key wie vorher)
    gdf.to_file(output_gpkg, layer=layer_key, driver="GPKG")

    print("Saved:", output_gpkg.name)

Gefundene Dateien: 3
 - master__helipads_POINTS_FOR_AGOL.gpkg
 - master__marinas_POINTS_FOR_AGOL.gpkg
 - master__towers_POINTS_FOR_AGOL.gpkg

INPUT : master__helipads_POINTS_FOR_AGOL.gpkg
OUTPUT: master__helipads_PUBLISH_READY.gpkg
Saved: master__helipads_PUBLISH_READY.gpkg

INPUT : master__marinas_POINTS_FOR_AGOL.gpkg
OUTPUT: master__marinas_PUBLISH_READY.gpkg
Saved: master__marinas_PUBLISH_READY.gpkg

INPUT : master__towers_POINTS_FOR_AGOL.gpkg
OUTPUT: master__towers_PUBLISH_READY.gpkg
Saved: master__towers_PUBLISH_READY.gpkg
